In [132]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import re

In [133]:
url_demo = ("https://www.makaan.com/bangalore-residential-property/buy-property-in-bangalore-city")
r = requests.get(url_demo)

In [134]:
r

<Response [200]>

In [135]:
r.text[:5000]

'<!doctype html> <html lang="en"><head><meta http-equiv="Content-type" content="text/html; charset=utf-8"><title>Property for Sale in Bangalore | 40477+ Bangalore Properties for sale</title><meta name="description" content="Find 40477+ Properties for sale in Bangalore on Makaan.com. Search &#10003;5997+ Houses/Villas for sale. &#10003;5909+ New Projects. &#10003;23189+ Apartments for sale. Book Now !"><meta name="keywords" content="Buy Bangalore Properties, Residential Property for sale in Bangalore, Property for sale in Bangalore, Bangalore Property Sale"><meta name="theme-color" content="#fff" id="themeColor"><meta content="origin" name="referrer"><meta name="p:domain_verify" content="55ce01b3ca93c05fd5a41439a23dd0d9"><meta name="fb:pages" content="155462194517712"><meta name="country" content="India"><meta name="og:type" content="website"><meta name="og:site_name" content="Makaan.com"><meta name="og:image:url" content="http://static.makaan.com/6/2/355/7975867.jpeg"><meta name="og:ti

In [136]:
data = soup(r.text)
house = data.find_all("li", class_="cardholder")
first = house[0]
first

<li class="cardholder" data-cardholder=""><div class="cardWrapper" data-listing-id="14173469" data-listing-projectid="1999926" data-module="listing" data-track-scroll="1"><script type="text/x-config">{"selector":"listing_14173469","sellerType":"BUILDER","builderId":102855,"builderName":"RBD Shelters","companyType":"BUILDER","companyId":1919379,"companyUserId":17347357,"backgroundColor":"#e2dded","textColor":"#fff","nameText":"R","companyName":"RBD Shelters","companyRating":0,"verificationDate":1622658600000,"serverTime":1622718102148,"listingUrl":"https://www.makaan.com/bangalore/rbd-shelters-meadows-1-in-anekal-city-14173469/835-sqft-plot","imageUrl":"https://static.makaan.com/1/1999926/291/meadows-1-landscaped-gardens-107195857.jpeg?width=460&amp;height=260","id":14173469,"propertyId":12710337,"projectId":1999926,"projectName":"Meadows 1","projectStatus":"Under Construction","mainImageId":"","imageCount":14,"latitude":12.74709225,"longitude":77.71852112,"localityName":"Anekal City","

In [137]:
price_first = first.find_all("td", class_="price")[0].text
price_first

' 15 L'

In [187]:
title=[]
location=[]
area=[]
builder=[]
building_status=[]
price=[]

In [188]:
#n=0
# starting from page 3 to 100 (url changes as per the page number from the third page)
for currPage in range(3,800):
    currPage=currPage+1
    url='https://www.makaan.com/bangalore-residential-property/buy-property-in-bangalore-city?page='+str(currPage)
    response=requests.get(url)
    page=soup(response.text, 'html.parser')
    # this will fetch all the info we need of a particular house in house_data
    house_data=data.find_all("li", class_="cardholder")
    
    # now for every element of house_data, we find the properties
    for house in house_data:
        typeOfHouse=house.find_all("a", class_="typelink")
        for i in typeOfHouse:
            x=i.text
            title.append(x)
            
            
            
        loc=house.find_all("a", class_="loclink")
        for i in loc:
            x=i.text
            x=x.split(',',)[0] # to get rid of "Bangalore" at the end of each location name
            location.append(x)
        
        
        
        area_sqft=house.find_all("td", class_="size")
        for i in area_sqft:
            x=i.text
            area.append(x)
            
        
        b=house.find_all("a", class_="seller-name")
        for i in b:
            x=i.text
            builder.append(x)
            
        
        
        cost=house.find_all("td", class_="price")
        for i in cost:
            x=i.text
            if 'L' in x:
                x1=x.split()[0]
                price.append(float(x1)) 
            elif 'Cr' in x:
                x2=x.split()[0]
                price.append(float(x2) * 100) 
            
            
            
        s=house.find_all("td", class_="val")
        for i in s:
            x=i.text
            building_status.append(x)

In [189]:
df=pd.DataFrame()

In [190]:
df['Title']=title 
df['Location']=location
df['Area Per Sqft']=area
df['Builder']=builder
df['Status']=building_status
df['Price in Lakhs']=price


In [191]:
housingCSV=df.to_csv("bangalore_housing_final.csv")

In [192]:
houseData=pd.read_csv("bangalore_housing_final.csv")

In [193]:
houseData.head(5)

,Unnamed: 0,Title,Location,Area Per Sqft,Builder,Status,Price in Lakhs
0,0,Residential Plot,Anekal City,835,RBD Shelters,New,15.00
1,1,1 BHK Apartment,Brookefield,739,Raja Housing,Under Construction,50.44
2,2,2 BHK Apartment,Electronic City Phase 1,1032,DS-MAX Properties,Under Construction,30.96
3,3,3 BHK Apartment,Krishnarajapura,1833,Togetherments Infra Pvt Ltd,Under Construction,128.00
4,4,2 BHK Apartment,Whitefield Hope Farm Junction,1238,SBB Group,Under Construction,75.84


In [194]:
houseData.shape

(15940, 7)

In [195]:
houseData.drop(columns='Unnamed: 0',axis=1,inplace=True)
houseData.head(5)

,Title,Location,Area Per Sqft,Builder,Status,Price in Lakhs
0,Residential Plot,Anekal City,835,RBD Shelters,New,15.00
1,1 BHK Apartment,Brookefield,739,Raja Housing,Under Construction,50.44
2,2 BHK Apartment,Electronic City Phase 1,1032,DS-MAX Properties,Under Construction,30.96
3,3 BHK Apartment,Krishnarajapura,1833,Togetherments Infra Pvt Ltd,Under Construction,128.00
4,2 BHK Apartment,Whitefield Hope Farm Junction,1238,SBB Group,Under Construction,75.84


In [196]:
houseData.describe()

,Area Per Sqft,Price in Lakhs
count,15940.000000,15940.000000
mean,1728.500000,96.533000
std,980.326956,71.521328
min,739.000000,15.000000
25%,953.250000,42.095000
50%,1455.000000,69.265000
75%,2331.250000,132.000000
max,4000.000000,288.000000
